# Switched Network

## Key Device: a Bridge

A bridge is a device that connects two groups of hosts:

<br>
<img src="images/18-simple-bridge.png" width="400">
<br>

The links on either side of the bridge are shared.

If A broadcasts a message to B, B will receive it since it is on the shared link.

The job of the bridge is to decide whether to broadcast a message over to the other side.

A simple bridge keeps a table of hosts connected to each of its ports:

```
Host | Port
-----------
  A  |  1
  B  |  1
  C  |  1
  X  |  2
  Y  |  2
  Z  |  2
```

With this table, the bridge can always decide whether it needs to forward a packet to the another port.

# Building the Table

For small static networks, the table could be hard coded.

For dynamic networks, the table is learned as packets are received.

The forwarding table starts empty. When the bridge receives a packet on one of its ports, an entry is added listing the host and its port.

Every entry has a Time To Live (TTL). If the TTL for an entry expires before another packet arrives from its host, that entry is dropped. 

# A More Complicated Switched Network

Many switched networks consist of switches with hosts connected to them.

<br>
<img src="images/19-learning-bridges.png" width="400">
<br>

Most network topologies are not acyclic in order to provide redundant paths. 

We don't want packets to get stuck in loops in the network.

We need to configure the forwarding tables of the network so that the network overall looks like an acyclic graph.

In other words, we want to use only a set of active links so that the graph looks like a tree.

<br>
<img src="images/20-learning-bridges-spanning.png" width="400">
<br>

This is exactly the problem of identifying a spanning tree on the graph of this network.

Every switch only has a local view of the network. It knows its ports and which packets it received on those ports.

Yet they all in concert need to configure their individual forwarding tables to have no cycles.

We will discuss a distributed algorithm for determing the spanning tree on network graphs.

## Distributed Algorithm for Determining a Spanning Tree over a Switched Network

At system boot, every switch needs to decide which ports to forward packets over.

No switch has a global view of the network, but they must still agree together which edges to use.

Strategy: All switches must decide 
- who the root is
- which preferred port to use to have the shortest path to root

They do this by exchanging configuration messages.

Configuration messages contain:
- The ID of the sender
- The ID of the switch the sender thinks is the root
- The distance in hops from the sender to the root

At the end of the process, the switch with the smallest ID will be the root of the network.

Every switch will track three pieces:
- who the root is
- how far away the root is
- which port to use to communicate towards the root

When a configuration message is received, a switch will update its configuration is:
- the message identifies a root with a smaller ID
    - Update who the root is and distance to the root.
- the message identifies the same root, but the sender is closer to the root
    - Update the port to use and distance to root
- the message identifies the same root and distance, but the sender has a smaller ID than the current preferred port
    - Update the preferred port to go through the switch with the smaller ID

Once a switch is no longer the root, it does not originate message it only forwards along the roots messaged (with an updated distance) along the preferred ports.

At the end of this, only the root is periodically sending messages.

If any switch stops receiving messages, it assumes the root is down and starts broadcasting configuration messages again.

The process will continue and a new spanning tree will be arrived at.

